In [1]:
print("hello")

hello


In [2]:
%pwd

'c:\\Users\\shasw\\OneDrive\\Desktop\\SIH\\MediBotics\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shasw\\OneDrive\\Desktop\\SIH\\MediBotics'

In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\shasw\anaconda3\envs\MediBotics\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shasw\anaconda3\envs\MediBotics\lib\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: The specified module could not be found. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
#Extract data from pdf
def load_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    doc=loader.load()
    return doc

In [7]:
data_from_pdf=load_file(data='Data/')

In [4]:
%pwd

'c:\\Users\\shasw\\OneDrive\\Desktop\\SIH\\MediBotics\\research'

In [5]:
import os
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\shasw\\OneDrive\\Desktop\\SIH\\MediBotics'

In [9]:
# data_from_pdf

In [16]:
def process_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
    texts = text_splitter.split_documents(documents)
    return texts


In [17]:
chunks=process_documents(data_from_pdf)
print("length of chunks:", len(chunks))

length of chunks: 8690


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
def download_hugging_embeddings():
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [5]:
embedding = download_hugging_embeddings()

c:\Users\shasw\anaconda3\envs\MediBotics\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shasw\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
PINECONE_API_KEY = os.getenv('Pinecone_API_Key')

In [15]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc=Pinecone(api_key=PINECONE_API_KEY)

index_name="medibotics"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",
                            region="us-east-1"
            )
    )